# **Modeling**

#### **Import Packages**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.ensemble import RandomForestClassifier


#### **Load the Data**

### **Neural Network**

### **Random Forrest**

In [ ]:
# Create a random forest classifier with 100 trees
rf = RandomForestClassifier(n_estimators=100, random_state=0)

# Fit the model to the data
rf.fit(X, y)